# <CENTER> Labb SH1015: Mätning av Z-bosonens massa med data från ATLAS-experimentet
<img src="https://cds.cern.ch/record/1631395/files/vp1_Htautau_1muon1electron_run204153_evt35369265.png">

## <CENTER> Introduktion
   
I denna labb får ni titta på riktig data från ATLAS! Experimentet har gjort en liten del av datan insamlad under 2012 tillgänglig för allmänheten. 
Ni kommer att analysera data med python, interaktivt, i denna notebook-miljö.
Målet är att mäta $Z$-bosonens massa genom att rekonstruera den *invarianta massan* av elektron-positron-par och myon-antimyon-par.

## <CENTER> $Z$-bosonen

Z-bosonen är en av Standardmodellens *kraftbärarpartiklar*, som tillsammans med $W$-bosonen förmedlar den svaga kärnkraften.
Den upptäcktes vid CERN 1983 m.h.a. acceleratorn Super-Proton-Synchrotron (då den starkaste acceleratorn, idag används den används som "föraccelerator" till Large Hadron Collider).

Det är en tung partikel (väger nästan lika mycket som en silveratom). 
Vi sparar ett referensvärde på dess massa (den bästa uppskattningen vi har idag), för att sedan kunna jämföra värdet det med ert uppmätta värde. 
Kolla upp värdena från Particle Data Group och spara de här för senare användning.

In [3]:
# referensvärden för Z-bosonens massa, från http://pdg.lbl.gov/2018/listings/rpp2018-list-z-boson.pdf
mZ_reference = ... # massa i GeV
mZ_unc_reference = ... # osäkerhet

Bosonen sönderfaller till två fermioner, t.ex. till en elektron och en positron, vilket representeras i följande Feynmandiagram:

## <CENTER> Händelser i ATLAS
    
Varje gång två protonhopar ($10^{11}$ protoner/hop) korsar varandra i ATLAS (var 25:e nanosekund) kolliderar ca 40 par. Partiklar (t.ex. en $Z$-boson) kan skapas ur energin i krocken och sönderfaller sedan om de är instabila. De stabila partiklarna flyger sedan ut genom detektorn och registreras av ATLAS. Varje sådan "bild" är ca 1 MB och kallas för ett "event" eller på svenska en "händelse". Datafilerna ni ska analysera är organiserade som tabeller där varje rad motsvarar en händelse och varje kolumn innehåller någon särkild information, t.ex. antalet registrerade elektroner och deras respektive energi.

## <CENTER> Invariant massa

Om en händelse innehåller t.ex. två elektroner, kan man beräkna den invarianta massan för dem. 
Om vi gör approximationen att elektronerna är masslösa fås invarianta massan som
$$
M^2 = E_1 E_2 - \vec{p_1}\cdot\vec{p_2} = 2 p_1 p_2 (1 - \cos\alpha),
$$
där $p$ står för rörelsemängd och $\alpha$ är vinkeln mellan elektronerna. 

I ATLAS mäts partiklars transversella rörelsemängd $p_\rm{T}$ och färdriktning relativt $z$-axeln (strålröret) med vinkeln $\theta$ och i $x-y$-planet med vinkeln $\phi$. Då ges invarianta massan av 
$$
M^2 = 2 p_\rm{T1} p_\rm{T2} (\cosh(\eta_1 - \eta_2) - \cos(\phi_1 - \phi_2))
$$
där variabeln "pseudorapiditet" $\eta = -\ln(\tan(\theta/2))$ använts.

Vi importerar C++-biblioteket ROOT som innehåller verktyg för att analysera och visualisera data.

In [2]:
import ROOT

Welcome to JupyROOT 6.14/04


## <CENTER> Data

Vi hämtar ATLAS open data från internet och laddar in det i ett ROOT-träd. Vi börjar med simulerad data, där en Z-boson skapats och sönderfallit till ett elektron-positron-par.

In [24]:
# Öppna en fil med simulerad data, innehållandes Z-bosoner som sönderfaller till elektron-positron-par (därav "Zee" i namnet)
f = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/MC/mc_147770.Zee.root")
tree = f.Get("mini") # hämta trädet. Varje "rad" i ett träd är en händelse

tree.Print("all") # denna rad visar varje gren ("kolumn") i trädet, alltså olika data som karakteriserar händelsen
for ievt in range(3): # kolla närmare på de tre första händelserna
    tree.GetEntry(ievt) # ladda händelse nr ievt
    print("Händelse {} :: Antalet leptoner = {}".format(ievt, tree.lep_n)) # "lep_n" är antalet leptoner, med leptoner
                                                                           # menar vi här elektroner eller myoner
    for ilep in range(tree.lep_n): # loopa över leptonerna
        print("        Lepton {} är av typ {} och har transversell rörelsemängd på {:.0f} GeV".format(ilep+1, tree.lep_type[ilep], tree.lep_pt[ilep]*1e-3))

Händelse 0 :: Antalet leptoner = 1
        Lepton 1 är av typ 11 och har transversell rörelsemängd på 30 GeV
Händelse 1 :: Antalet leptoner = 2
        Lepton 1 är av typ 11 och har transversell rörelsemängd på 93 GeV
        Lepton 2 är av typ 11 och har transversell rörelsemängd på 20 GeV
Händelse 2 :: Antalet leptoner = 2
        Lepton 1 är av typ 11 och har transversell rörelsemängd på 43 GeV
        Lepton 2 är av typ 11 och har transversell rörelsemängd på 37 GeV
******************************************************************************
*Tree    :mini      : 4-vectors + variables required for scaling factors     *
*Entries :  7500000 : Total =      2134734070 bytes  File  Size =  966057844 *
*        :          : Tree compression factor =   2.21                       *
******************************************************************************
*Br    0 :runNumber : runNumber/I                                            *
*Entries :  7500000 : Total  Size=   30006369 bytes

Nu testar vi att plotta distributionen av den transversella rörelsemängden för elektroner i de första 1000 händelserna.

In [22]:
h_pt = ROOT.TH1F("h_pt", "; Transverse momentum [GeV]; Number of electrons", 15, 15., 90.) # histogram med 15 binnar
nevents=1000
ievt=0
for evt in tree: # loopa över händelser
    ievt+=1
    if ievt >= nevents: break
    for ilep in range(evt.lep_n):
        if evt.lep_type[ilep] == 11: # vi vill bara rita elektroner (typ=11)
            h_pt.Fill(evt.lep_pt[ilep]*1e-3) # fyll histogram

Warning in <TDavixFile::Append>: Replacing existing TH1: h_pt (Potential memory leak).


In [16]:
# "magic"-kommando för visualisering
%jsroot on 

In [23]:
# öppna en canvas att rita på
canvas = ROOT.TCanvas("Canvas","a first way to plot a variable",800,600)
# rita histogrammet
h_pt.Draw("e1p")
# öppna canvasen
canvas.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: Canvas


Den uppritade figuren är interaktiv: testa att klicka runt (använd högerklick) och scrolla. Figuren kan sparas i png-format genom att högerklicka mitt på canvasen.

I nästa notebook börjar vi med själva uppgiften, där vi ska titta på faktisk data!